<a href="https://colab.research.google.com/github/arbouria/Notas-Aprendizaje-y-Comportamiento-Adaptable-I/blob/main/Simulador_Bayesiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, Button, HBox, VBox, Output, Label
from IPython.display import display
from scipy.stats import beta # Para la distribución Beta

# --- Variables Globales para el Simulador Bayesiano ---
# Parámetros iniciales de la distribución Beta (equivalente a 2 ensayos previos)
# alpha_prior y beta_prior representan el número efectivo de "éxitos" y "fracasos" previos
alpha_prior_initial = 1.0
beta_prior_initial = 1.0

# Contadores de ensayos observados (éxitos y fracasos)
observed_successes = 0
observed_failures = 0

# Objeto de salida para mostrar el gráfico
bayesian_output = Output()

# --- Función para Calcular y Graficar la Distribución Posterior ---
def update_bayesian_plot(initial_belief_strength_slider_value):
    global observed_successes, observed_failures

    # Mapear el slider 'fuerza de creencia inicial' a los parámetros Beta iniciales
    # Esto permite que el usuario ajuste un prior más o menos "fuerte" o "débil"
    # Una forma simple es usar un factor de escala, digamos que el slider representa un prior basado en 10 observaciones efectivas
    effective_initial_observations = 10
    alpha_0 = initial_belief_strength_slider_value * effective_initial_observations
    beta_0 = (1 - initial_belief_strength_slider_value) * effective_initial_observations

    # Si alpha_0 o beta_0 son cero, dales un valor mínimo para evitar errores con la distribución Beta
    alpha_0 = max(0.01, alpha_0)
    beta_0 = max(0.01, beta_0)


    # Calcular los parámetros de la distribución Beta posterior
    # alpha_posterior = alpha_prior_initial + observed_successes
    # beta_posterior = beta_prior_initial + observed_failures
    alpha_posterior = alpha_0 + observed_successes
    beta_posterior = beta_0 + observed_failures


    # Calcular la media de la distribución Beta posterior (nuestra creencia actualizada)
    posterior_mean = alpha_posterior / (alpha_posterior + beta_posterior)

    # Generar puntos para graficar la distribución Beta
    x = np.linspace(0, 1, 100)
    y_prior = beta.pdf(x, alpha_0, beta_0) # Distribución prior
    y_posterior = beta.pdf(x, alpha_posterior, beta_posterior) # Distribución posterior

    # Crear el gráfico
    fig, ax = plt.subplots(figsize=(8, 5))
    ax.plot(x, y_prior, label=f'Creencia Inicial (Media: {alpha_0 / (alpha_0 + beta_0):.2f})', color='orange', linestyle='--')
    ax.plot(x, y_posterior, label=f'Creencia Actualizada (Media: {posterior_mean:.2f})', color='blue')
    ax.axvline(posterior_mean, color='red', linestyle=':', label=f'Media Posterior: {posterior_mean:.2f}')

    ax.set_title('Simulación de Actualización de Creencias (Bayesiana)')
    ax.set_xlabel('Probabilidad de Asociación (Campana -> Comida)')
    ax.set_ylabel('Densidad de Probabilidad')
    ax.legend()
    ax.grid(True)
    ax.set_ylim(bottom=0) # Asegurar que el eje y empiece en 0

    # Limpiar el output anterior y mostrar el nuevo gráfico
    with bayesian_output:
        bayesian_output.clear_output(wait=True)
        display(fig)
    plt.close(fig) # Cierra la figura para evitar que se muestre dos veces

# --- Callbacks para los Botones de Ensayos ---
def on_success_click(b):
    global observed_successes
    observed_successes += 1
    success_label.value = f'Éxitos (Campana y Comida): {observed_successes}'
    update_bayesian_plot(initial_belief_slider.value)

def on_failure_click(b):
    global observed_failures
    observed_failures += 1
    failure_label.value = f'Fracasos (Campana y NO Comida): {observed_failures}'
    update_bayesian_plot(initial_belief_slider.value)

def on_reset_bayesian_click(b):
    global observed_successes, observed_failures
    observed_successes = 0
    observed_failures = 0
    success_label.value = 'Éxitos (Campana y Comida): 0'
    failure_label.value = 'Fracasos (Campana y NO Comida): 0'
    update_bayesian_plot(initial_belief_slider.value)

# --- Widgets ---
initial_belief_slider = FloatSlider(
    min=0.01, max=0.99, step=0.01, value=0.5,
    description='Creencia Inicial P(Comida|Campana):',
    orientation='horizontal'
)
initial_belief_slider.observe(lambda change: update_bayesian_plot(change.new), names='value')

success_button = Button(description='Campana y Comida (Éxito)')
failure_button = Button(description='Campana y NO Comida (Fracaso)')
reset_bayesian_button = Button(description='Reiniciar Ensayos Bayesianos')

success_button.on_click(on_success_click)
failure_button.on_click(on_failure_click)
reset_bayesian_button.on_click(on_reset_bayesian_click)

success_label = Label(value=f'Éxitos (Campana y Comida): {observed_successes}')
failure_label = Label(value=f'Fracasos (Campana y NO Comida): {observed_failures}')

# Layout de los widgets
bayesian_controls = VBox([
    initial_belief_slider,
    HBox([success_button, success_label]),
    HBox([failure_button, failure_label]),
    reset_bayesian_button
])

# --- Display del Simulador ---
print("## Simulador Bayesiano: Actualización de Creencias en el Aprendizaje Asociativo")
print("Este simulador muestra cómo tu creencia sobre la probabilidad de que una campana prediga la comida se actualiza con cada ensayo. Ajusta tu creencia inicial y luego observa los ensayos.")
display(HBox([bayesian_controls, bayesian_output]))

# Inicializar el gráfico
update_bayesian_plot(initial_belief_slider.value)

## Simulador Bayesiano: Actualización de Creencias en el Aprendizaje Asociativo
Este simulador muestra cómo tu creencia sobre la probabilidad de que una campana prediga la comida se actualiza con cada ensayo. Ajusta tu creencia inicial y luego observa los ensayos.
